# Sefl-discover reasoning guideline

Using Self-Discover generate Reasoning structure to solve the problem using LLama3-70b & use this as a guideline to solve the problem using a 7b model

### Loading modules

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q datasets
!pip install together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 43.9 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.3.0 which is incompatible.


In [3]:
import os
from datasets import load_dataset, Dataset


## Load Dataset

In [4]:
# model_name = "meta-llama/Llama-2-7b-chat-hf"
from datasets import load_dataset, Dataset

dataset_name = "aqua_rat"
test_dataset = load_dataset(dataset_name, split="test")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/97467 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/254 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/254 [00:00<?, ? examples/s]

In [5]:
for i in range(1):
  print(test_dataset[i])

{'question': 'A car is being driven, in a straight line and at a uniform speed, towards the base of a vertical tower. The top of the tower is observed from the car and, in the process, it takes 10 minutes for the angle of elevation to change from 45° to 60°. After how much more time will this car reach the base of the tower?', 'options': ['A)5(√3 + 1)', 'B)6(√3 + √2)', 'C)7(√3 – 1)', 'D)8(√3 – 2)', 'E)None of these'], 'rationale': 'Explanation :\nLet the height of the building be h. Initially, he was at an angle of 450. tan 45 = h/distance between car and tower. h = distance between car and tower (since tan 45 = 1).\nNow, after 10 minutes, it travelled a certain distance, and angle changed to 600.\ntan 60 = h/x x = h/√3\nSo, in 10 minutes, it has travelled a distance of h – x = h - h/√3.\n10 minutes = h *( 1 – 1√3)\nh can be travelled in 10 / (1 – 1√3).\nTo travel a distance of x, which is h/√3, it takes :\nh = 10 / (1 – 1/√3)\nh / √3 = 10/ √3 * (1 – 1/√3). Multiply numerator and denom

## Generate Reasoning structure using LLama 3 70B Chat Model

In [6]:
import os
from together import Together
TOGETHER_API_KEY = "3f06bb7945192b6e58e9d4acd1d265fe51a1478015fb5cac5d67c3ca274b3c94"
client = Together(api_key=TOGETHER_API_KEY)


In [7]:
import re
# from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score, f1_score

def query_llama3(prompt):
    """
    Inference from the LLaMA-3 Chat (70B) using together API.

    :param prompt: A string representing the constructed prompt
    :return: A formatted string representing the complete api response for the input prompt
    """
    response = client.chat.completions.create(
          model="meta-llama/Llama-3-70b-chat-hf", #TODO: model type
          messages=[{"role": "user", "content": prompt}],
      )
    # print(f'Together API response is: {response.choices[0].message.content}')
    return response.choices[0].message.content


In [8]:

reasoning_modules = [
    "1. How could I devise an experiment to help solve that problem?",
    "2. Make a list of ideas for solving this problem, and apply them one by one to the problem to see if any progress can be made.",
    #"3. How could I measure progress on this problem?",
    "4. How can I simplify the problem so that it is easier to solve?",
    "5. What are the key assumptions underlying this problem?",
    "6. What are the potential risks and drawbacks of each solution?",
    "7. What are the alternative perspectives or viewpoints on this problem?",
    "8. What are the long-term implications of this problem and its solutions?",
    "9. How can I break down this problem into smaller, more manageable parts?",
    "10. Critical Thinking: This style involves analyzing the problem from different perspectives, questioning assumptions, and evaluating the evidence or information available. It focuses on logical reasoning, evidence-based decision-making, and identifying potential biases or flaws in thinking.",
    "11. Try creative thinking, generate innovative and out-of-the-box ideas to solve the problem. Explore unconventional solutions, thinking beyond traditional boundaries, and encouraging imagination and originality.",
    #"12. Seek input and collaboration from others to solve the problem. Emphasize teamwork, open communication, and leveraging the diverse perspectives and expertise of a group to come up with effective solutions.",
    "13. Use systems thinking: Consider the problem as part of a larger system and understanding the interconnectedness of various elements. Focuses on identifying the underlying causes, feedback loops, and interdependencies that influence the problem, and developing holistic solutions that address the system as a whole.",
    "14. Use Risk Analysis: Evaluate potential risks, uncertainties, and tradeoffs associated with different solutions or approaches to a problem. Emphasize assessing the potential consequences and likelihood of success or failure, and making informed decisions based on a balanced analysis of risks and benefits.",
    #"15. Use Reflective Thinking: Step back from the problem, take the time for introspection and self-reflection. Examine personal biases, assumptions, and mental models that may influence problem-solving, and being open to learning from past experiences to improve future approaches.",
    "16. What is the core issue or problem that needs to be addressed?",
    "17. What are the underlying causes or factors contributing to the problem?",
    "18. Are there any potential solutions or strategies that have been tried before? If yes, what were the outcomes and lessons learned?",
    "19. What are the potential obstacles or challenges that might arise in solving this problem?",
    "20. Are there any relevant data or information that can provide insights into the problem? If yes, what data sources are available, and how can they be analyzed?",
    "21. Are there any stakeholders or individuals who are directly affected by the problem? What are their perspectives and needs?",
    "22. What resources (financial, human, technological, etc.) are needed to tackle the problem effectively?",
    "23. How can progress or success in solving the problem be measured or evaluated?",
    "24. What indicators or metrics can be used?",
    "25. Is the problem a technical or practical one that requires a specific expertise or skill set? Or is it more of a conceptual or theoretical problem?",
    "26. Does the problem involve a physical constraint, such as limited resources, infrastructure, or space?",
    "27. Is the problem related to human behavior, such as a social, cultural, or psychological issue?",
    "28. Does the problem involve decision-making or planning, where choices need to be made under uncertainty or with competing objectives?",
    "29. Is the problem an analytical one that requires data analysis, modeling, or optimization techniques?",
    "30. Is the problem a design challenge that requires creative solutions and innovation?",
    "31. Does the problem require addressing systemic or structural issues rather than just individual instances?",
    "32. Is the problem time-sensitive or urgent, requiring immediate attention and action?",
    "33. What kinds of solution typically are produced for this kind of problem specification?",
    "34. Given the problem specification and the current best solution, have a guess about other possible solutions."
    "35. Let’s imagine the current best solution is totally wrong, what other ways are there to think about the problem specification?"
    "36. What is the best way to modify this current best solution, given what you know about these kinds of problem specification?"
    "37. Ignoring the current best solution, create an entirely new solution to the problem."
    #"38. Let’s think step by step."
    "39. Let’s make a step by step plan and implement it with good notation and explanation."
]



In [9]:

# STAGE 1

def select_reasoning_modules(task_description, options, reasoning_modules):
    """
    Step 1: SELECT relevant reasoning modules for the task.
    """
    prompt = "You are an mathematical assistant that helps users with the Algebra questions. You are asked a multi-choice question along with the answer options - A, B, C, D, E. \n\n"
    prompt += f"Given the task: {task_description} with options: {options}, which of the following reasoning modules are relevant? Do not elaborate on why.\n\n" + "\n".join(reasoning_modules) + "\n\n"
    prompt += "Select the reasoning modules.\n\n"
    # print("Selecting reasoning modules for the task...")
    # print(prompt)
    selected_modules = query_llama3(prompt)
    return selected_modules
    # return prompt

def adapt_reasoning_modules(selected_modules, task_example, options):
    """
    Step 2: ADAPT the selected reasoning modules to be more specific to the task.
    """
    prompt = "You are an mathematical assistant that helps users with the Algebra questions. You are asked a multi-choice question along with the answer options - A, B, C, D, E. \n\n"
    prompt += f"Without working out the full solution, adapt the following reasoning modules to be specific to our task:\n"
    prompt += f"Here are the reasoning modules:\n{selected_modules}\n\n"
    prompt += f"Here is our Task Description:\n{task_example} with options: {options}\n"
    prompt += "Provide the adapted reasoning modules.\n\n"
    # print("Adapting reasoning modules for the task...")
    # print(prompt)
    adapted_modules = query_llama3(prompt)
    return adapted_modules
    # return prompt

def implement_reasoning_structure(adapted_modules, task_description, options):
    """
    Step 3: IMPLEMENT the adapted reasoning modules into an actionable reasoning structure.
    """
    prompt = "You are an mathematical assistant that helps users with the Algebra questions. You are asked a multi-choice question along with the answer options - A, B, C, D, E. \n\n"
    prompt += f"Without working out the full solution, create an actionable reasoning structure for the task using these adapted reasoning modules:\n"
    prompt += f"Here are the adapted reasoning modules:\n{adapted_modules}\n\n"
    prompt += f"Here is our Task Description:\n{task_description} with options: {options}\n"
    prompt += "Provide the actionable reasoning structure and do not provide the final solution.\n\n"
    # print("Implementing reasoning structure for the task...")
    # print(prompt)
    reasoning_structure = query_llama3(prompt)
    return reasoning_structure
    # return prompt



In [10]:
# generate reasoning structure for test_dataset

# test_dataset {question, options, rationale, correct}

import pandas as pd
from tqdm.notebook import tqdm
import time

def generate_reasoning_structure(test_dataset, reasoning_structure_csv_file):
  # Load existing data from CSV
  existing_data = pd.read_csv(reasoning_structure_csv_file)
  start_point = len(existing_data)
  print(existing_data[start_point - 1:])
  print('Start point is ' + str(start_point))

  test_df = test_dataset.to_pandas()
  batch_size = 10
  for i in tqdm(range(start_point, len(test_df), batch_size), unit='batch'):
    new_rows = []
    # Process each batch
    for j in tqdm(range(batch_size)):
      if i + j >= len(test_df):
        break

      question = test_df.loc[i+j, 'question']
      options = test_df.loc[i+j, 'options']
      rationale = test_df.loc[i+j, 'rationale']
      correct = test_df.loc[i+j, 'correct']
      # print(question)

      # if i+j in set([125]):
      #   new_row = {'question': question, 'options': options, 'rationale': rationale, 'correct': correct,
      #             'selected_modules': None, 'adapted_modules': None, 'reasoning_structure' : None}
      #   new_rows.append(new_row)
      #   continue

      # Handle API timeout errors
      try:
          # select reasoning modules
          selected_modules = select_reasoning_modules(question, options, reasoning_modules)
          # adapt reasoning modules
          adapted_modules = adapt_reasoning_modules(selected_modules, question, options)
          # implement reasoning structure
          reasoning_structure = implement_reasoning_structure(adapted_modules, question, options)

          # Create dictionary representing new row
          new_row = {'question': question, 'options': options, 'rationale': rationale, 'correct': correct,
                    'selected_modules': selected_modules, 'adapted_modules': adapted_modules, 'reasoning_structure' : reasoning_structure}
          new_rows.append(new_row)
      except Exception as error:
          # Upload to Google Drive before exiting
          print(error)
          upload_to_drive(reasoning_structure_csv_file)
          print("API Error occurred. Saving progress and exiting.")
          new_data = pd.DataFrame(new_rows)
          new_data.to_csv(reasoning_structure_csv_file, index=False, mode='a', header=not os.path.exists(reasoning_structure_csv_file))
          return
      print(f"Inference done for question {i + j + 1}")

    new_data = pd.DataFrame(new_rows)
    new_data.to_csv(reasoning_structure_csv_file, index=False, mode='a', header=not os.path.exists(reasoning_structure_csv_file))
    time.sleep(5)
  print(f"Updated {reasoning_structure_csv_file} with new inferences.")


def load_reasoning_structure(reasoning_structure_csv_file):
  if os.path.exists(reasoning_structure_csv_file):
    inferenced_data = pd.read_csv(reasoning_structure_csv_file)
    return inferenced_data
  else:
    return None



In [12]:
folder_path = "/content/drive/My Drive/AQua_RAT/"
!mkdir -p folder_path
reasoning_structure_csv_file = "self_discover_reasoning_structure.csv"
reasoning_structure_csv_file = os.path.join(folder_path, reasoning_structure_csv_file)

# Function to authenticate Google Drive and upload CSV
def upload_to_drive(csv_file):
  # file_path = os.path.join(folder_path, csv_file)
  if not os.path.exists(csv_file):
    # Create the file if it doesn't exist
    with open(file_path, 'w') as f:
        # Write header to the CSV file
        f.write("question,options,rationale,correct,selected_modules,adapted_modules,reasoning_structure\n")
    print(f"File {csv_file} created in {folder_path}")
  else:
    print(f"File {csv_file} already exists in {folder_path}")
    return

upload_to_drive(reasoning_structure_csv_file)

File /content/drive/My Drive/AQua_RAT/self_discover_reasoning_structure.csv already exists in /content/drive/My Drive/AQua_RAT/


In [42]:
generate_reasoning_structure(test_dataset, reasoning_structure_csv_file)


Empty DataFrame
Columns: [question, options, rationale, correct, selected_modules, adapted_modules, reasoning_structure]
Index: []
Start point is 0


  0%|          | 0/26 [00:00<?, ?batch/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Inference done for question 1
Inference done for question 2
Inference done for question 3
Inference done for question 4
Inference done for question 5
Inference done for question 6
Inference done for question 7
Inference done for question 8
Inference done for question 9
Inference done for question 10


  0%|          | 0/10 [00:00<?, ?it/s]

...

Inference done for question 241
Inference done for question 242
Inference done for question 243
Inference done for question 244
Inference done for question 245
Inference done for question 246
Inference done for question 247
Inference done for question 248
Inference done for question 249
Inference done for question 250


  0%|          | 0/10 [00:00<?, ?it/s]

Inference done for question 251
Inference done for question 252
Inference done for question 253
Inference done for question 254
Updated /content/drive/My Drive/AQua_RAT/self_discover_reasoning_structure.csv with new inferences.


Map:   0%|          | 0/254 [00:00<?, ? examples/s]

KeyError: 'selected_modules'

In [13]:
reasoning_structure = load_reasoning_structure(reasoning_structure_csv_file)

In [14]:
test_dataset = Dataset.from_pandas(reasoning_structure)

In [15]:
test_dataset

Dataset({
    features: ['question', 'options', 'rationale', 'correct', 'selected_modules', 'adapted_modules', 'reasoning_structure'],
    num_rows: 254
})

## Evaluate CodeLLama model on AQua-RAT dataset using reasoning structure as guideline

In [16]:
def query_codellama(prompt):
    """
    Inference from the Codellama-7b-Instruct-hf using together API.

    :param prompt: A string representing the constructed prompt
    :return: A formatted string representing the complete api response for the input prompt
    """
    response = client.chat.completions.create(
          model="codellama/CodeLlama-7b-Instruct-hf", #TODO: model type
          messages=[{"role": "user", "content": prompt}],
          max_tokens=2048,
          temperature= 0.7,
          top_p= 0.7,
          top_k= 50,
          repetition_penalty= 1.5,
      )
    # print(f'Together API response is: {response.choices[0].message.content}')
    return response.choices[0].message.content

# STAGE 2

def execute_reasoning_structure(reasoning_structure, task_instance, options):
    """
    Execute the reasoning structure to solve a specific task instance.
    """
    prompt = "You are an mathematical assistant that helps users with the Algebra questions. You are asked a multi-choice question along with the answer options - A, B, C, D, E. "
    prompt += f"Using the following reasoning structure: {reasoning_structure}\n\n"
    prompt += "Solve the provided task and choose the correct option. Follow the below instructions while responding.\n"
    prompt += "1) Use the provided reasoning structure as a guideline to solve the problem. \n"
    prompt += "2) Provide intermediate steps for solving the problems. \n"
    prompt += "3) Always end the solution by providing your final answer using the format : \"The correct answer option is ({})\" \n"
    prompt += f"Here is our Task Description:\n{task_instance} with \n"
    prompt += f"Options: {options} \n\n"

    # print("Executing reasoning structure...")
    # print(prompt)
    solution = query_codellama(prompt)
    return solution
    # return prompt

In [18]:
import pandas as pd

folder_path = "/content/drive/My Drive/AQua_RAT/"
!mkdir -p folder_path
file_name = "ZeroShotWithSelfDiscoverGuidelinePredictions_AQUARAT3.csv"
file_path = os.path.join(folder_path, file_name)

# Function to authenticate Google Drive and upload CSV
def upload_to_drive(csv_file):
  if not os.path.exists(file_path):
    # Create the file if it doesn't exist
    with open(file_path, 'w') as f:
        # Write header to the CSV file
        f.write("question,options,rationale,selected_modules,adapted_modules,reasoning_structure,true_answer,predicted_answer,output\n")
    print(f"File {file_name} created in {folder_path}")
  else:
    print(f"File {file_name} already exists in {folder_path}")
    return

upload_to_drive(file_path)

File ZeroShotWithSelfDiscoverGuidelinePredictions_AQUARAT3.csv already exists in /content/drive/My Drive/AQua_RAT/


## Evaluation

In [19]:
import time


def extract_answer_option(text):
    """
    Extract the correct answer option from the reasoning
    """
    # Regular expression patterns to match the given cases
    pattern_exp = '\s*\(?\s*([A-Ea-e])\s*\)'
    correct_answer_patterns = [
        r'The\s*correct\s*answer\s*option\s*is',
        r'The\s*correct\s*answer\s*is',
        r'The\s*correct\s*answer\s*is\s*option'
    ]

    patterns = [re.compile(pattern  + pattern_exp, re.IGNORECASE) for pattern in correct_answer_patterns]

    # Search for the patterns in the text in the specified order
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            return match.group(1).upper()

    # If no match is found, return None
    return None


def get_predicted_zero_shot_data(test_df, csv_file):
  upload_to_drive(csv_file)
  # Load existing data from CSV
  existing_data = pd.read_csv(csv_file)
  start_point = len(existing_data)
  print(existing_data[start_point - 1:])
  print('Start point is ' + str(start_point))


  batch_size = 10
  for i in tqdm(range(start_point, len(test_df), batch_size), unit='batch'):
    new_rows = []
    # Process each batch
    for j in tqdm(range(batch_size)):
      if i + j >= len(test_df):
        break

      question = test_df.loc[i+j, 'question']
      options = test_df.loc[i+j, 'options']
      rationale = test_df.loc[i+j, 'rationale']
      correct = test_df.loc[i+j, 'correct']
      selected_modules = test_df.loc[i+j, 'selected_modules']
      adapted_modules = test_df.loc[i+j, 'adapted_modules']
      reasoning_structure = test_df.loc[i+j, 'reasoning_structure']
      # print(question)

      # if i+j in set([125]):
      #   new_row = {'question': question, 'options': options, 'rationale': rationale, 'selected_modules': selected_modules, 'adapted_modules': adapted_modules,
      #              'reasoning_structure' : reasoning_structure, 'true_answer': correct, 'predicted_answer': None}
      #   new_rows.append(new_row)
      #   continue

      # Handle API timeout errors
      try:
          output = execute_reasoning_structure(reasoning_structure, question, options)
          predicted_answer = extract_answer_option(output)

          # Create dictionary representing new row
          new_row = {'question': question, 'options': options, 'rationale': rationale, 'selected_modules': selected_modules, 'adapted_modules': adapted_modules,
                    'reasoning_structure' : reasoning_structure, 'true_answer': correct, 'predicted_answer': predicted_answer, 'output': output}
          new_rows.append(new_row)
      except Exception as error:
          # Upload to Google Drive before exiting
          print(error)
          upload_to_drive(csv_file)
          print("API Error occurred. Saving progress and exiting.")
          new_data = pd.DataFrame(new_rows)
          new_data.to_csv(csv_file, index=False, mode='a', header=not os.path.exists(csv_file))
          return
      print(f"Inference done for question {i + j + 1}")

    new_data = pd.DataFrame(new_rows)
    new_data.to_csv(csv_file, index=False, mode='a', header=not os.path.exists(csv_file))
    time.sleep(5)
  print(f"Updated {csv_file} with new inferences.")


In [20]:
get_predicted_zero_shot_data(pd.DataFrame(test_dataset), file_path)

File ZeroShotWithSelfDiscoverGuidelinePredictions_AQUARAT3.csv created in /content/drive/My Drive/AQua_RAT/
Empty DataFrame
Columns: [question, options, rationale, selected_modules, adapted_modules, reasoning_structure, true_answer, predicted_answer, output]
Index: []
Start point is 0


  0%|          | 0/26 [00:00<?, ?batch/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Inference done for question 1
Inference done for question 2
Inference done for question 3
Inference done for question 4
Inference done for question 5
Inference done for question 6
Inference done for question 7
Inference done for question 8
Inference done for question 9
Inference done for question 10


  0%|          | 0/10 [00:00<?, ?it/s]

...

Inference done for question 231
Inference done for question 232
Inference done for question 233
Inference done for question 234
Inference done for question 235
Inference done for question 236
Inference done for question 237
Inference done for question 238
Inference done for question 239
Inference done for question 240


  0%|          | 0/10 [00:00<?, ?it/s]

Inference done for question 241
Inference done for question 242
Inference done for question 243
Inference done for question 244
Inference done for question 245
Inference done for question 246
Inference done for question 247
Inference done for question 248
Inference done for question 249
Inference done for question 250


  0%|          | 0/10 [00:00<?, ?it/s]

Inference done for question 251
Inference done for question 252
Inference done for question 253
Inference done for question 254
Updated /content/drive/My Drive/AQua_RAT/ZeroShotWithSelfDiscoverGuidelinePredictions_AQUARAT3.csv with new inferences.


## Aggregated Metrics

In [26]:
from sklearn.metrics import accuracy_score, f1_score
def get_data_from_csv(file_path):
  if os.path.exists(file_path):
    inferenced_data = pd.read_csv(file_path)
    return inferenced_data
  else:
    return None

def compute_metrics(true_labels, pred_labels):
    """
    Load and return the dataset for inference
    """
    accuracy = accuracy_score(true_labels, pred_labels)
    f1 = f1_score(true_labels, pred_labels, average='macro') #TOOD: check macro

    return {
        'accuracy': accuracy,
        'f1_score': f1
    }

def get_evaluation_metrics(inferenced_data):
  true_answers = inferenced_data['true_answer'].tolist()
  predicted_answers = inferenced_data['predicted_answer'].tolist()
  accuracy = accuracy_score(true_answers, predicted_answers)
  f1 = f1_score(true_answers, predicted_answers, average='macro')
  # rouge_L, rouge_1, bertscore = inferenced_data['rouge_L'].mean(), inferenced_data['rouge_1'].mean(), inferenced_data['bertscore'].mean()
  print(accuracy)
  print(f1)
  # print(rouge_L)
  # print(rouge_1)
  # print(bertscore)

inferenced_data = get_data_from_csv(file_path)

get_evaluation_metrics(inferenced_data)

0.051181102362204724
0.06451277480689245


In [ ]:
# inferenced_data = get_data_from_csv(file_path)
# print(inferenced_data,file_path)
#get_evaluation_metrics(inferenced_data)

                                              question  \
0    A car is being driven, in a straight line and ...   
1    The original price of an item is discounted 22...   
2    Find out which of the following values is the ...   
3    If the probability that Stock A will increase ...   
4    A trader sold an article at a profit of 20% fo...   
..                                                 ...   
249  The capacity of a tank of dimensions (8 m × 6 ...   
250  The population of a city is 5265526. If there ...   
251  It takes a worker 9 minutes to drive from home...   
252  Two friends are eating a jar full of candies. ...   
253  A grocery sells a bag of ice for $1.25, and ma...   

                                               options  \
0    ['A)5(√3 + 1)', 'B)6(√3 + √2)', 'C)7(√3 – 1)',...   
1    ['A)$61', 'B)$65', 'C)$67.40', 'D)$70', 'E)$78...   
2               ['A)36', 'B)15', 'C)17', 'D)5', 'E)7']   
3    ['A)0.22', 'B)0.26', 'C)0.37', 'D)0.46', 'E)0....   
4        ['A)